In [1]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil
import logging
logger = logging.getLogger(str(os.getpid()))

from wattile.data_reading import read_dataset_from_file, _concat_data_from_files
PROJECT_DIRECTORY = Path().resolve().parent.parent

PROJECT_DIRECTORY = C:\Users\JKIM4\Anaconda3\envs\wattile\Lib\site-packages\wattile


In [2]:
import copy

# reading configs

In [3]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# main configs file
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)
##################################################################################
# code testing configs file
# with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_input"]["exp_dir"] = str(PROJECT_DIRECTORY / exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "data" / "Synthetic Site")

configs

{'data_input': {'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\data\\Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power',
  'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir'},
 'data_output': {'exp_dir': 'exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'fea

In [4]:
def _get_dataset_config(configs):
    """Get dataset config as dataframe

    :param configs: configs
    :type configs: dict
    :return: dataset config
    :rtype: pd.DataFrame
    """
    dataset_dir = Path(configs["data_input"]["data_dir"])
    configs_file_inputdata = dataset_dir / configs["data_input"]["data_config"]

    logger.info(
        "Pre-process: reading input data summary json file from {}".format(
            configs_file_inputdata
        )
    )

    with open(configs_file_inputdata, "r") as read_file:
        configs_input = json.load(read_file)
        df_inputdata = pd.DataFrame(configs_input["files"])

    # converting date time column into pandas datetime (raw format based on ISO 8601)
    df_inputdata["start"] = pd.to_datetime(
        df_inputdata.start, format="t:%Y-%m-%dT%H:%M:%S%z", exact=False, utc=True
    )
    df_inputdata["end"] = pd.to_datetime(
        df_inputdata.end, format="t:%Y-%m-%dT%H:%M:%S%z", exact=False, utc=True
    )

    df_inputdata["path"] = str(dataset_dir) + "/" + df_inputdata["filename"]

    return df_inputdata, configs_input

# reading final selected predictors

In [5]:
df_inputdata, configs_input = _get_dataset_config(configs)
configs_input

{'dates': {'start': 't:2021-12-01T00:00:00-07:00 Denver',
  'end': 't:2021-12-08T00:00:00-07:00 Denver'},
 'predictors': [{'site': 'Synthetic Site',
   'column': 'Synthetic Weather Station Dew Point Temperature',
   'id': 'r:278f8943-6a199bd7 Synthetic Weather Station Dew Point Temperature',
   'description': 'Synthetic Weather Station Dew Point Temperature',
   'unit': 'Â°F',
   'pv': False},
  {'site': 'Synthetic Site',
   'column': 'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'id': 'r:278f79c0-03da5abc Synthetic Weather Station Diffuse Horizontal Irradiance',
   'description': 'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'unit': 'W/mÂ²_irr',
   'pv': False},
  {'site': 'Synthetic Site',
   'column': 'Synthetic Weather Station Direct Normal Irradiance',
   'id': 'r:278f79c0-4fe536b4 Synthetic Weather Station Direct Normal Irradiance',
   'description': 'Synthetic Weather Station Direct Normal Irradiance',
   'unit': 'W/mÂ²_irr',
   'pv': False},
  {'

In [6]:
configs_input["predictors"]

[{'site': 'Synthetic Site',
  'column': 'Synthetic Weather Station Dew Point Temperature',
  'id': 'r:278f8943-6a199bd7 Synthetic Weather Station Dew Point Temperature',
  'description': 'Synthetic Weather Station Dew Point Temperature',
  'unit': 'Â°F',
  'pv': False},
 {'site': 'Synthetic Site',
  'column': 'Synthetic Weather Station Diffuse Horizontal Irradiance',
  'id': 'r:278f79c0-03da5abc Synthetic Weather Station Diffuse Horizontal Irradiance',
  'description': 'Synthetic Weather Station Diffuse Horizontal Irradiance',
  'unit': 'W/mÂ²_irr',
  'pv': False},
 {'site': 'Synthetic Site',
  'column': 'Synthetic Weather Station Direct Normal Irradiance',
  'id': 'r:278f79c0-4fe536b4 Synthetic Weather Station Direct Normal Irradiance',
  'description': 'Synthetic Weather Station Direct Normal Irradiance',
  'unit': 'W/mÂ²_irr',
  'pv': False},
 {'site': 'Synthetic Site',
  'column': 'Synthetic Weather Station Dry Bulb Temperature',
  'id': 'r:278f79c0-8a8f26b0 Synthetic Weather Stati

In [7]:
list_ref = ["Synthetic Weather Station Diffuse Horizontal Irradiance","Synthetic Weather Station Relative Humidity"]

In [8]:
# create list of final predictors with the same format in data_config
predictor_final_temp = []
for entry in configs_input["predictors"]:
    if entry["column"] in list_ref:
        predictor_final_temp.append(entry)
predictor_final = {}
predictor_final["column"] = predictor_final_temp
predictor_final

{'column': [{'site': 'Synthetic Site',
   'column': 'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'id': 'r:278f79c0-03da5abc Synthetic Weather Station Diffuse Horizontal Irradiance',
   'description': 'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'unit': 'W/mÂ²_irr',
   'pv': False},
  {'site': 'Synthetic Site',
   'column': 'Synthetic Weather Station Relative Humidity',
   'id': 'r:278f79c0-e7db4305 Synthetic Weather Station Relative Humidity',
   'description': 'Synthetic Weather Station Relative Humidity',
   'unit': '%RH',
   'pv': False}]}

In [9]:
# # create list of final predictors with the same format in data_config
# predictor_final = {}
# predictor_final["column"] = []
# for entry in configs_input["predictors"]:
#     if entry["column"] in list_ref:
#         if entry["column"] not in predictor_final["column"]:
#             predictor_final["column"].append(entry)
# predictor_final

In [10]:
# # create separate json file including final predictors
# file_prefix = Path(configs["data_output"]["exp_dir"])
# path = os.path.join(file_prefix, "predictors config.json")
# with open(path, "w") as fp:
#     json.dump(predictor_final, fp, indent=1)